Hello everyone, this is the first thing that I'm putting on the kaggle and from now I will continously updating it.<br>
So as a beginner it is difficult to get this done I have referred to others expert's kernel's which is [here](https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python) and [here](https://www.kaggle.com/masumrumi/a-detailed-regression-guide-with-house-pricing) and there are so many other great kernels to look into.<br>
The main aim behind this kernel to make understand basics to beginner and give little understanding about exploratory data analysis approach.<br>



In [ ]:
#importing required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from scipy.stats import skew
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# loading data
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test  = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
train.head()

In [ ]:
# checking the dimension of datasets
train.shape, test.shape

In [ ]:
# ckeck the columns
train.columns

linear regression model makes a good amount of **assumptions** for the data you provide

1. Linearity 
2. No noise 
3. No collinearity
4. Normal distribution
5. Scale

## Missing data

### Handling missing data from both test and train datasets

_Handling missing values is an essential part of data cleaning and preparation process_.<br>
***np.nan, None and NaT (for datetime64 types)*** _are standard missing value for Pandas_.<br>
_Not all missing values come in nice and clean **np.nan or None format** instead there may be some characters like `??`or `--`,etc_.

In [ ]:
# missing data from train dataset
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total,percent],axis=1,keys=['Total','Percent'])
missing_data.head(30)

We will delete the variable when there is more then **15% missing data**, according to this 'PoolQC','MiscFeature','Alley','Fence','FireplaceQu','LotFrontage' will get deleted.

'GarageType','GarageYrBlt', 'GarageFinish', 'GarageQual','GarageCond' have the same number of missing data so this missing data refers to the same set of observation so we delete this variables and we have considered 'GarageCars' which expressed the most of the information about the garage.

Same logic goes for 'BsmtExposure', 'BsmtFinType1','BsmtQual','BsmtCond','BsmtFinType2'.
'MasVnrArea' and 'MasVnrType' haave strong corelation with 'YearBuilt' and 'OverallQual' which we have already considered.

Finally 'Electrical' have only one null value so we replace it with its mode.


In [ ]:
# Dealing with missing data
train = train.drop((missing_data[missing_data['Total']>1]).index,1)
train['Electrical'] = train['Electrical'].fillna(train['Electrical'].mode()[0])
train.isnull().sum().max()

In [ ]:
# After deleting the columns 
train.columns

In [ ]:
# missing data from test dataset
total = test.isnull().sum().sort_values(ascending=False)
percent = (test.isnull().sum()/test.isnull().count()).sort_values(ascending=False)
missing_data1 = pd.concat([total,percent],axis=1,keys=['Total','Percent'])
missing_data1.head(40)

As we did before for **train** data same logic applies here for **test** data.

In [ ]:
test = test.drop((missing_data1[missing_data1['Total']>4]).index,1)

In [ ]:
# missing data from test dataset
total = test.isnull().sum().sort_values(ascending=False)
percent = (test.isnull().sum()/test.isnull().count()).sort_values(ascending=False)
missing_data1 = pd.concat([total,percent],axis=1,keys=['Total','Percent'])
missing_data1.head(40)

For `categorical variables` and `numerical variable` we are just filling the **Null** values with most frequent(**mode**) from specified columns.

In [ ]:
null_features = (missing_data1[missing_data1['Total']>0]).index
null_features

In [ ]:
for feature in null_features:
    test[feature] = test[feature].fillna(test[feature].mode()[0])

In [ ]:
# check again if there is any null values
test.isnull().sum().max()

## Analysing SalePrice 

In [ ]:
#Descriptive statistics summary
train['SalePrice'].describe()

In [ ]:
#histogram
sns.distplot(train['SalePrice']);

##### Let's focus on 1st asumption

In [ ]:
# correlation matrix
corrmat = train.corr()
f, ax = plt.subplots(figsize=(12,9))
sns.heatmap(corrmat,vmax=.8,square = True);

We used **heatmap** here, so we can get the overview of all the features relationship 

In [ ]:
# most correlated features 
corrmat = train.corr()
top_corr_features = corrmat.index[abs(corrmat['SalePrice'])>0.5]
plt.figure(figsize=(10,10))
sns.heatmap(train[top_corr_features].corr(),annot = True);
top_corr_features

This are the features most **correlated** with the 'SalePrice'.

**OverallQual, GrLivArea, GarageCars and TotalBsmtSF** are strongly correlated with the 'SalePrice'.

**GarageCars** and **GarageArea** are also some of the most strongly correlated variables.

Same goes for **TotalBsmtSF** and **1stFloor**.

## Scatter plot between 'SalePrice' and its correlated Variables 

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(train[cols], size = 2.5)
plt.show();

## Fixing Skewness

##### Lets focus on 4th assumption, and that is that predictors and target variable should follow a gaussian distribution. 

Here we are using **Log Transform**

In [ ]:
# differentiate between numerical and categorical varibles
categorical_features = train.select_dtypes(include = ["object"]).columns
numerical_features = train.select_dtypes(exclude = ["object"]).columns

In [ ]:
# taking numerical dataset and categorical datasets separately 
train_num = train[numerical_features]
train_cat = train[categorical_features]

In [ ]:
train_cat.shape,train_num.shape

In [ ]:
# checkin skewmess of all features
skewness = train_num.apply(lambda x: skew(x))
skewness.sort_values(ascending=False)

##### we are selecting features where skewness is greater than 0.5 to fix their skewness

In [ ]:
skewness = skewness[abs(skewness)>0.5]
skewness.index

In [ ]:
# applying log tranform
#train_num[skewness.index] = np.log1p(train_num[skewness.index])

In [ ]:
train_cat.head()

In [ ]:
# using get_dummies here it is used for data manipulation. It converts categorical data into dummy or indicator variables
train_cat = pd.get_dummies(train_cat)

In [ ]:
train_cat.shape

In [ ]:
# concatenating train_num (numerical variable) and train_cat (categorical variable)
train1 = pd.concat([train_cat,train_num],axis=1)
train1.shape

## Skewness of test data

#### same applies here as we did for train data

In [ ]:
# differentiate between numerical and categorical varibles
categorical_features = test.select_dtypes(include = ["object"]).columns
numerical_features = test.select_dtypes(exclude = ["object"]).columns

test_num = test[numerical_features]
test_cat = test[categorical_features]

In [ ]:
test_num.shape,test_cat.shape

In [ ]:
# finding skewness of all features
skewness = test_num.apply(lambda x: skew(x))
skewness.sort_values(ascending=False)

In [ ]:
# we are selecting features where skewness is greater than 0.5 to fix their skewness
#skewness = skewness[abs(skewness)>0.5]
#len(skewness.index)

In [ ]:
# applying log tranform
#test_num[skewness.index] = np.log1p(test_num[skewness.index])

In [ ]:
test_cat.head()

In [ ]:
test_cat = pd.get_dummies(test_cat)

In [ ]:
test_cat.head()

In [ ]:
test1 = pd.concat([test_cat,test_num],axis=1)


In [ ]:
test1.shape

## Outliers

### Using 3 standard deviation to remove outliers 

**`Standard Deviation `** means simply it shows you how far away data point from the mean.<br>
If our dataset is normally distributed then most of the data fall under `1 standard deviation nearly 68% of data`.<br>
Here we will use **`3 standard deviation` to `remove outliers`**. 
Nearly **`99.7%`** of values are within **`3 standard deviation`** of the mean and the data `outside the 3 standard deviation` we consider them as `outlier's`. 

In [ ]:
# set minimum and maximum threshold values to detect ouliers using standard deviation
min_threshold = train1.SalePrice.mean() - 3*train1.SalePrice.std()
max_threshold = train1.SalePrice.mean() + 3*train1.SalePrice.std()

In [ ]:
min_threshold,max_threshold

In [ ]:
# removing the outlier's from dataset
train1 = train1[(train1.SalePrice<max_threshold) & (train1.SalePrice)>min_threshold]

In [ ]:
train1.shape,test1.shape

## Modeling 

In [ ]:
# importing all the required library for modeling here we are going to use statsmodels 
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [ ]:
cols = [col for col in train1.columns if col not in test1.columns]
cols.remove('SalePrice')
train1 = train1.drop(cols,axis=1)

Now our task will be training and testing the data, so we need to drop the `SalePrice` from the training dataset and will assign it to the `y`

In [ ]:
# assining the required data to the respective variables  
X = train1.drop(['SalePrice'],axis=1)
y = train1['SalePrice']

In [ ]:
#X1 = sm.add_constant(X)
#test2 = sm.add_constant(test1)

In [ ]:
# checking shapes
test1.shape,train1.shape

In [ ]:
train1.shape

In [ ]:
#model = sm.OLS(y, X).fit()
#predictions = model.predict(test2)
#print("ROOT MEAN SQUARED ERROR : ",math.sqrt(sum((y-predictions)**2)/len(y)))


In [ ]:
# parity plot for statsmodels OLS
''''plt.scatter(predictions,y,color='blue')
plt.title('Linear Regression')
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.plot([10.5,13.5],[10.5,13.5],c='red')
plt.show()'''

trying to optimize model with **XGBoost** stands for **extreme gradient boosting**

In [ ]:
from xgboost import XGBRegressor
X_train,X_valid,y_train,y_valid = train_test_split(X,y,random_state=0)
model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)
predictions1 = model.predict(X_valid)
print("ROOT MEAN SQUARED ERROR : ",math.sqrt(sum((y_valid-predictions1)**2)/len(y_valid)))

In [ ]:
# parity plot 
plt.scatter(predictions1,y_valid,color='blue')
plt.title('Linear Regression')
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.plot([50000,700000],[50000,700000],c='red')
plt.show()

The above scatterplot comparing the actual values against predicted values in an easy understandable way.

In [ ]:
# lets now test for the test set
predictions = model.predict(test1)

In [ ]:
submission = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
final_submission = pd.DataFrame({'Id':submission['Id'],'SalePrice':predictions})
final_submission.to_csv('submission1.csv', index=False)